# Can we find triples about new entities?

In [134]:
# get classes & instance predictions
import pandas as pd

file_classes = './data/output_tables_instance_v1/class_decision.csv'
file_instances = './data/output_tables_instance_v1/instance_correspondences.csv'
file_schemas = './data/output_tables_instance_v1/schema_correspondences.csv'

file_classes = './data/t2d_instance_v1/gs_class.csv'
file_instances = './data/t2d_instance_v1/gs_instance.csv'
file_schemas = './data/t2d_instance_v1/gs_property.csv'

dir_tables = './data/t2d_instance_v1/tables/'

In [139]:
classes = pd.read_csv(file_classes, header=None, index_col = 0)
if list(classes.index)[0] == 0:
    classes.set_index(1, inplace=True)
else:
    del classes[2]
classes.columns = ['name']
# classes.head()

In [136]:
schemas = pd.read_csv(file_schemas, header=None)
schemas['table'] = schemas[0].map(lambda s: s.split('~')[0])
schemas['col'] = schemas[0].map(lambda s: int(s.split('~Col')[1]))
del schemas[0]
schemas.set_index('table', inplace=True)
# schemas.head()

In [137]:
instances = pd.read_csv(file_instances, header=None)
instances['table'] = instances[0].map(lambda s: s.split('~')[0])
instances['row'] = instances[0].map(lambda s: int(s.split('~Row')[1]))
del instances[0]
instances.set_index('table', inplace=True)
# instances.head()

In [140]:
import os, glob

TYPE_URI = 'type'

def parse_cell(cell):
    cell = str(cell)
    if cell[0] == '{' and cell[-1] == '}':
        for part in cell[1:-1].split('|'):
            yield part
    elif cell == '&nbsp;':
        return
    else:
        yield cell

for fname in glob.glob('%s/*.*' % dir_tables):
    name = os.path.basename(fname)
    table = pd.read_csv(fname)
    
    d = pd.DataFrame(instances.loc[name])
    d.set_index('row', inplace=True)
    
    schema = {t.col:t._1 for t in schemas.loc[[name]].itertuples()}
    types = set(c for cs in classes.loc[[name]].itertuples() for c in cs.name.split(','))
    
    for tup in table.itertuples():
        i = tup.Index
        instance = d.loc[i][1] if i in d.index else None
        if not instance:
            row_id = '%s~Row%s' % (name, i)
            for t in types:
                print(row_id, TYPE_URI, t, sep='\t')
            for i, cell in enumerate(list(tup)[1:]):
                if i in schema:
                    for c in parse_cell(cell):
                        print(row_id, schema[i], c, sep='\t')
        break
#     break



28079336_2_2139598421036200532.csv~Row0	type	Company
28079336_2_2139598421036200532.csv~Row0	http://www.w3.org/2000/01/rdf-schema#label	company name
28079336_2_2139598421036200532.csv~Row0	http://dbpedia.org/ontology/industry	industry
32812626_0_5428724146873158593.csv~Row0	type	Hospital
32812626_0_5428724146873158593.csv~Row0	http://www.w3.org/2000/01/rdf-schema#label	6th medical group - macdill air force base
32812626_0_5428724146873158593.csv~Row0	http://dbpedia.org/ontology/circle	macdill air force base
32812626_0_5428724146873158593.csv~Row0	http://dbpedia.org/ontology/bedCount	0
69537082_0_7789694313271016902.csv~Row0	type	Country
69537082_0_7789694313271016902.csv~Row0	http://www.w3.org/2000/01/rdf-schema#label	world
69537082_0_7789694313271016902.csv~Row0	http://dbpedia.org/ontology/populationTotal	6446131400
41194422_0_7231546114369966811.csv~Row0	type	Country
41194422_0_7231546114369966811.csv~Row0	http://www.w3.org/2000/01/rdf-schema#label	world
41194422_0_723154611436996681